The notebook is equivalent for each GloVe embedding (standard or Twitter)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd 
data = pd.read_csv('/content/drive/MyDrive/Thesis/Data/Filled Datasets/Second dataset (Kaggle)/Preprocessed/combined_train_val.csv',header=0)
test = pd.read_csv('/content/drive/MyDrive/Thesis/Data/Filled Datasets/Second dataset (Kaggle)/Preprocessed/Constraint_Test_fullprep_with_labels.csv')

In [3]:
test

,id,tweet,label
0,1,daily update published states reported k tests...,0
1,2,alfalfa cure covid,1
2,3,president trump asked would catch coronavirus,1
3,4,states reported deaths still seeing solid nati...,0
4,5,sixth time global health emergency declared in...,0
...,...,...,...
2134,2136,state wise details total confirmed cases till ...,0
2135,2137,tonight midnight onwards disaster management a...,1
2136,2138,new cases plateau enough yo lagos rivers ft fa...,0
2137,2139,one stop shop resources section communicating ...,0


In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sn
import pickle

%matplotlib inline

#Import module to split the datasets
from sklearn.model_selection import train_test_split
# Import modules to evaluate the metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc

In [5]:
y_train = data["label"]
X_train = data["tweet"]
y_test = test["label"]
X_test = test["tweet"]

In [14]:
# We just need to run this code once, the function glove2word2vec saves the Glove embeddings in the word2vec format 
# that will be loaded in the next section
from gensim.scripts.glove2word2vec import glove2word2vec

#glove_input_file = glove_filename
word2vec_output_file = 'glove.twitter.27B.100d.txt.word2vec'
glove2word2vec("/content/drive/MyDrive/Thesis/Data/Embedding files/glove.6B.100d.txt", word2vec_output_file)

(400001, 100)

In [15]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

**Vectorization**

In [16]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('virus')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.split()
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [17]:
import numpy as np
# Set a word vectorizer
vectorizer = Word2VecVectorizer(model)
# Get the sentence embeddings for the train dataset
Xtrain = vectorizer.fit_transform(X_train)
Ytrain = y_train
# Get the sentence embeddings for the test dataset
Xtest = vectorizer.transform(X_test)
Ytest = y_test

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 1 / 8558
Numer of samples with no words found: 0 / 2139


oversampling

In [18]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
Xtrain, Ytrain = smote.fit_resample(Xtrain,Ytrain)

For evaluation metrics: 

In [19]:
# Create the confussion matrix
def plot_confussion_matrix(y_test, y_pred):
    ''' Plot the confussion matrix for the target labels and predictions '''
    cm = confusion_matrix(y_test, y_pred)

    # Create a dataframe with the confussion matrix values
    df_cm = pd.DataFrame(cm, range(cm.shape[0]),
                  range(cm.shape[1]))
    #plt.figure(figsize = (10,7))
    # Plot the confussion matrix
    sn.set(font_scale=1.4) #for label size
    sn.heatmap(df_cm, annot=True,fmt='.0f',annot_kws={"size": 10})# font size
    plt.show()
    
# ROC Curve
# plot no skill
# Calculate the points in the ROC curve
def plot_roc_curve(y_test, y_pred):
    ''' Plot the ROC curve for the target labels and predictions'''
    fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
    roc_auc= auc(fpr,tpr)

    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

**The SVM Model**

In [20]:
from sklearn.svm import SVC
SVM = SVC()
SVM.fit(Xtrain, Ytrain)


SVC()

In [21]:
y_pred = SVM.predict(Xtest)

print(metrics.classification_report(Ytest, y_pred,  digits=5))
print(confusion_matrix(Ytest, y_pred))

              precision    recall  f1-score   support

           0    0.88655   0.86595   0.87613      1119
           1    0.85660   0.87843   0.86738      1020

    accuracy                        0.87190      2139
   macro avg    0.87157   0.87219   0.87175      2139
weighted avg    0.87227   0.87190   0.87196      2139

[[969 150]
 [124 896]]
